# Análise de Vendas de Rede de Lojas

## Índice da Análise

1. Business Problem
    * Starting Context
        * Business Understanding
2. Starting Phase
    * CRISP-DS
3. Analysis Phase
    * Data Collection
    * Data Cleaning
        * Feature Extraction
    * Data Analysis
        * Descriptive Data Analysis
        * Hypotheses Mindmap
        * Exploratory Data Analysis
4. Model Phase
    * Feature Engineering
        * Data Preparation
        * Feature Selection
    * Model Building
        * Train-Test Splitting
        * Model Selection
            * Baseline Training
            * Cross Validation Training
            * Models Performance
    * Model Evaluation
        * Model Hyperparameter Fine Tuning
        * Metrics Interpretation
            * Business Metrics
            * Model Metrics
5. Deployment Phase
    * Visualization and Dashboard
        * Performance Assessment
        * Model Performance
            * Baseline vs Model Performance
            * Model Performance in Business
        * Business Performance Gain
    * API development
        * Prediction Class
        * API Handler
        * API Tester
    * Web App
        * Frontend

# Business Problem

## Starting Context

"contexto inicial aqui"

##### Business Understanding

É possível entender o problema de negócio fazendo apenas 4 perguntas, estas são:

* Qual a Motivação?
    - Qual o contexto?
* Qual a causa raiz do problema?
    - Porque fazer uma análise de vendas e não outra coisa?
* Quem é o dono do problema?
    - Quem precisa dessa solução? Gerentes, CFO? Quem vai nos cobrar?
* Qual o formato da solução de deploy?
    - Granularidade:
        - Qual o alcance esperado para se fazer essa análise?
        - Análise feita semanal, diária, mensal, por loja, por cidade etc
    - Tipo de Análise:
        - Será regressão, classificação, clustering, etc
    - Potenciais Métodos:
        - Podemos supor que vai ser usado qual algoritmo? random forest, regressão linear, KNN, Cross Validation, etc
    - Formato de deploy:
        - Aplicação web, aplicação mobile, bot do Telegram, Docker, Heroku, AWS, etc

No nosso caso, as respostas para as perguntas acima são:

* Qual a motivação?
* Qual a causa raiz do problema?
* Quem é o dono do problema?
* Qual o formato da solução de deploy?
    - Granularidade:
    - Tipo de Análise:
    - Potenciais Métodos:
    - Formato de deploy:
        - Pedrições acessadas via celular.
        - Utilizar API do Telegram.
        - Predições acessadas via web
        - Aplicação web

# Starting Phase

Na fase inicial do projeto importamos as bibliotecas necessárias, fazemos as configurações iniciais do notebook e de algumas bibliotecas.

Com ela bem feita vamos ter tudo pronto para seguir a análise bem configurada e reprodutível para outras pessoas.

## Libraries

In [ ]:
try:
    # Fundamental Analytics libraries
    import pandas as pd
    import seaborn as sns
    import numpy as np
    from matplotlib import pyplot as plt
    from scipy import stats as sts
    from sklearn.preprocessing import RobustScaler, MinMaxScaler, LabelEncoder
    # Toolbox libraries
    import time
    import math
    import pickle
    import random
    import requests
    import inflection
    import warnings
    from operator import itemgetter as pick
    from datetime import datetime, timedelta
    from loguru import logger as log
    from IPython.display import Image
    from IPython.core.display import HTML
    # Model libraries
    from boruta import BorutaPy
    from sklearn.linear_model import LinearRegression, Lasso
    from sklearn.ensemble import RandomForestRegressor
    from xgboost import XGBRegressor
    import xgboost
    from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
    # Initial configurations
    warnings.filterwarnings('ignore')
    warnings.simplefilter(action = 'ignore', category = FutureWarning)
    xgboost.set_config(verbosity = 0)
    # Info about imports
    print("Import successful")
    #log.info("Import successful")
    
except Exception as e:
    print("Error while importing libraries: ", "/n", e)
    #log.error("Error while importing libraries: ", e) 

In [ ]:
Start = datetime.now() # time object

## CRISP-DS

In [ ]:
Image(filename='..\\..\\Assets\\Images\\Data Science Project Cycle.png', width=600, height=550)

Cross-Industry Standard Process for Data Science é um método cíclico de desenvolvimento
- Desde o primeiro ciclo vamos ter uma versão end-to-end da solução
- Vamos passar pelas etapas da análise várias vezes
- Temos mais velocidade na entrega de valor
- Conseguimos mapear os possíveis problemas

##### Procedimento padrão dos projetos de Data Science

In [ ]:
Image(filename='..\\..\\Assets\\Images\\Data Science Project Guide.png')

## Dataset

#### Files


Descrição dos arquivos de entrada: 

#### Data fields

Descrição das colunas do arquivo CSV que serão utilizadas no projeto.

# Analysis Phase

In [ ]:
Image(filename='..\\..\\Assets\\Images\\Analysis Phase.png', width=250, height=100)

## Data Collection

In [ ]:
Image(filename='..\\..\\Assets\\Images\\Data Collection.png', width=200, height=100)

Como neste projeto não vamos usar comunicação com API ou SQL, vamos carregar os dados a partir de um arquivo .CSV

### Loading Data

In [ ]:
try:
    df_sales_raw = pd.read_csv('..\\..\\..\\Data\\Dataset\\Raw\\train.csv', low_memory=False)
    df_stores_raw = pd.read_csv('..\\..\\..\\Data\\Dataset\\Raw\\store.csv', low_memory=False)
    print("Loading successful")
    df_raw = pd.merge(df_sales_raw, df_stores_raw, how = "left", on = "Store")
    print("Merge successful")
    #log.info("Merge successful")
except Exception as e:
    print("Error while merging: ", "/n", e)
    #log.error("Error while merging: ", e) 

## Data Cleaning

In [ ]:
Image(filename='..\\..\\Assets\\Images\\Data Cleaning.png', width=200, height=100)

### Data Cleaning Checkpoint

Fazendo um checkpoint, isolanmos os resultados obtidos nesta seção dentro dela e evitamos propagações de erros que requeiram a reexecução do notebook inteiro.

In [ ]:
df1 = df_raw.copy()
print("Checkpoint successful")

In [ ]:
df1.head()

### Rename Columns

Vamos mudar as colunas para nomes mais significativos, visto que a partir dos dados brutos os nomes vem ideais para quem desenvolveu seu armazenamento, mas não para quem vai usar os dados numa análise futura.

Com isso os nomes ficam mais intuitivos e mais fluídos de entender ao longo da análise.

In [ ]:
col_df, colunas = get_column_names(df1)
col_df

In [ ]:
snakecase = lambda column: inflection.underscore(column) 
Colunas_new = list(map(snakecase, colunas))
df1.columns = Colunas_new
# df1.columns
col_df, colunas = get_column_names(df1)
col_df

### Data Dimensions

Descobrir o tamanho o Dataset e conhecer as dimensões de com o que estamos trabalhando.

### Data Types

Verificar os tipos de dados e se necessitam de alguma transformação para tornar o processamento mais eficiente ou mesmo possível.

### Missing values

Vamos verificar se há dados faltantes no dataset.

In [ ]:
missing_values, missing_columns_names = get_broadview_miss_val(df1)
missing_values

In [ ]:
fig, ax = plt.subplots(figsize=(12, 8))

sns.heatmap(df1.isnull(), yticklabels=False, cbar=False, cmap='magma', ax=ax)
plt.show()

Existem três maneiras de tratar nossos dados faltantes:
- Descartando completamente os dados faltantes
- Utilizando o próprio comportamento da coluna para substituir os dados faltantes, utilizando de média ou mediana
- Entendimento do negócio, utilizando algumas regras que podem ter passado despercebidas para substituir os dados faltantes a partir de um método mais específico para este dataset

In [ ]:
missing_columns_names

Esta é uma das partes mais críticas da análise pois se os dados faltantes forem muitos, o método de substituição deles pode ser determinante no sucesso ou fracasso do modelo nas seções mais abaixo.


Conferência final dos valores faltantes:

In [ ]:
missing_values1, missing_columns_names1 = get_broadview_miss_val(df1)
missing_values1

### Type Conversion

Após algumas transformações, os tipos de dados das colunas podem mudar sem sabermos, então vamos verificar o estado atual das colunas e checar se precisamos fazer alguma conversão.

In [ ]:
get_dataset_types(df1)

In [ ]:
df1.head()

In [ ]:
get_dataset_types(df1)

### Feature Extraction

### Variable Filtering

Vamos filtrar as variáveis para deixar o dataset mais leve e de acordo com as possíveis restrições de negócio que encontrarmos.

In [ ]:
df1.head()

#### Row Filtering

#### Column Filtering

## Data Analysis

In [ ]:
Image(filename='..\\..\\Assets\\Images\\Data Analysis.png', width=200, height=100)

### Data Analysis Checkpoint

Fazendo um checkpoint, copiamos o dataframe para uma nova variável, isolando os resultados obtidos nesta seção dentro dela e evitando propagações de erros que requeiram a reexecução do notebook inteiro.

In [ ]:
df2 = df1.copy()
print("Checkpoint successful")

In [ ]:
df2.head()

### Descriptive Data Analysis

Na análise descritiva vamos entender

In [ ]:
get_dataset_types(df2)

In [ ]:
num_attributes = df2.select_dtypes(include=['int64','float64'])
cat_attributes = df2.select_dtypes(exclude=['int32','int64','float64', 'datetime64[ns]'])

In [ ]:
num_attributes.head()

In [ ]:
cat_attributes.head()

#### Numerical Attributes

Vamos analisar os atributos numéricos:

In [ ]:
get_num_statistics_metrics(num_attributes)

#### Categorical Attributes

Vamos analisar os atributos categóricos:

In [ ]:
get_unique_cat_values(cat_attributes)

### Hypotheses Mindmap

Para chegarmos na Análise Exploratória de Dados e sabermos por qual caminho vamos nos guiar, vamos fazer uma lista de hipóteses a partir de três perguntas para analisar na próxima parte do projeto:

- Qual o fenômeno modelado?
- Quais são os agentes que atuam sobre o fenômeno de interesse?
- Quais são os atributos dos agentes?

In [ ]:
Image(filename='..\\..\\Assets\\Images\\Hypotheses Mindmap.png', width=900, height=500)

#### Hypotheses Questions

- Qual o fenômeno modelado?
- Quais são os agentes que atuam sobre o fenômeno de interesse?

- Quais são os atributos dos agentes?

#### Selected Hypotheses List

Esta é a lista final de hipóteses que vamos procurar confirmar ou falsear com a análise exploratória de dados, a partir das conclusões dessas hipóteses vamos ter uma idéia melhor de como vamos construir o modelo de previsão de vendas.

### Exploratory Data Analysis

Vamos listar primeiro quais os objetivos que queremos alcançar com a análise exploratória de dados.

#### Objectives

- Ganhar experiência de negócio
- Validar hipóteses de negócio
- Perceber quais variáveis são importantes para o modelo
- Gerar Insigths sobre o negócio 

##### Insights

Os insights de dados referem-se à compreensão profunda que um indivíduo ou empresa obtém ao analisar seus dados sobre um problema específico de negócio.

Essa compreensão profunda ajuda empresas a tomarem melhores decisões do que aquelas que se baseiam somente no instinto.

Insights podem ser gerados de duas formas:
- Surpresa
    - Uma conclusão nova surge através dos dados
- Quebra de crenças
    - Quando uma crença empírica sobre o negócio é refutada, e provada que na verdade era o inverso ou que era completamente inválida e sem base sólida

##### Processes

Quais são os processos que vamos utilizar para analisar os dados?

- Análise Univariada
- Análise Bivariada
- Análise Multivariada

Qual o objetivo de cada processo?

- Análise Univariada
    - Como é essa variável?
    - Mínimos, máximos, distribuição, range
- Análise Bivariada
    - Como essa variável impacta na variável alvo?
    - Correlação, validação de hipóteses
- Análise Multivariada
    - Como as variáveis se relacionam?
    - Correlação, validação de hipóteses

#### Univariate Analysis

Vamos analisar as variáveis de forma univariada, ou seja, vamos analisar apenas uma variável por vez, sem relação com outras.

##### Target Variable

Vamos analisar a variável alvo, ou seja, a variável que queremos prever.

In [ ]:
target = ""
sns.distplot(df2[target]).set_title(f'Distribution of {target}')

In [ ]:
sns.distplot(df2[target], kde=False, bins=30).set_title(f'Distribution of {target}')

##### Numerical Variables

Vamos analisar as características das variáveis numéricas.

In [ ]:
num_attributes = df2.select_dtypes(include=['int64','float64'])
num_attributes.head()

In [ ]:
fig = plt.figure(figsize = (30,15))
ax = fig.gca()
num_attributes.hist(ax = ax,bins=25)
plt.show()

Conclusões nessa fase:


Quanto as outras, precisamos compará-las com outras variáveis para tirar conclusões mais consistentes.

##### Categorical Variables

Vamos analisar as características das variáveis categóricas.

#### Bivariate Analysis

A partir daqui começamos a tirar conclusões mais consistentes sobre as nossas hipóteses. Vamos testar uma por uma e ver quais se confirmam.

##### H1

##### H2

##### H3

##### H4

##### H5

#### Hypotheses Validation

In [ ]:
conclusions =  [['H1', 'Falsa', 'Baixa', 'Sim'],  
                ['H2', 'Verdadeira', 'Média', 'Não'],
                ['H3', 'Verdadeira', 'Alta', 'Não'],
                ['H4', 'Verdadeira', 'Média', 'Possível'] ]

In [ ]:
Hypotheses = get_analysis_conclusions(conclusions, columns_included = False)
Hypotheses

#### Multivariate Analysis

##### Numerical Attributes

In [ ]:
num_attributes.head()

In [ ]:
correlation = num_attributes.corr(method='pearson')
sns.heatmap(correlation, annot=True)
plt.show()

##### Categorical Attributes

In [ ]:
cat_attributes.head()

In [ ]:
cat = df2.select_dtypes(include='object')
cat.head()

# Model Phase

In [ ]:
Image(filename='..\\..\\Assets\\Images\\Model Phase.png', width=250, height=100)

## Feature Engineering

In [ ]:
Image(filename='..\\..\\Assets\\Images\\Feature Engineering.png', width=200, height=100)

### Feature Engineering Checkpoint

Fazendo um checkpoint, copiamos o dataframe para uma nova variável, isolando os resultados obtidos nesta seção dentro dela e evitando propagações de erros que requeiram a reexecução do notebook inteiro.

In [ ]:
df3 = df2.copy()
print("Checkpoint successful")

In [ ]:
df3.head()

### Objectives

O Feature Engineering serve para transformar os dados em um formato que pode ser utilizado pelo modelo.

O Feature Engineering é o último processo que lida com o dataset antes dele ser utilizado pelo modelo, e é composto principalmente de duas partes:
- Data Preparation
- Feature Selection

### Data Preparation

Na fase de Data Preparation, o dataset é tratado para que possamos utilizá-lo no modelo, essa fase é composta de quatro partes:
- Tratamento de variáveis numéricas
    - Normalização
        - Rescala o centro dos dados para zero com desvio padrão igual a 1
        - Funciona melhor para dados que já possuem uma distribuição normal
    - Rescaling
        - Rescala os dados para um intervalo de valores entre 0 e 1
        - Funciona melhor para dados que não possuem uma distribuição normal
- Tratamento de variáveis categóricas
    - Encoding
        - Transforma dados categóricos em dados numéricos
        - Existem vários tipos de encoding, como:
            - One-Hot Encoding
            - Label Encoding
            - Ordinal Encoding
- Tratamento de variáveis cíclicas
    - Transformação de Seno e Cosseno
        - Preserva a natureza cíclica dos dados
- Tratamento da variável resposta
    - Rescalam os valores da variável resposta de modo a ter uma distribuição mais próxima da normal
        - Transformação logarítmica
        - Transformação Box-Cox
        - Transformação Square Root
        - Transformação Cube Root

#### Numerical Variable Treatment

##### Normalization

Se tivermos dados que possuem uma distribuição normal, podemos normalizar os dados para que o centro seja zero e o desvio padrão seja igual a 1.

Se pela análise univariada um dado não possui uma distribuição normal, eles não se encaixa no processo de normalização.

##### Rescaling

Se tivermos dados que não possuem uma distribuição normal, podemos rescalar os dados para que o intervalo deles fique entre 0 e 1.

Tipos de rescaling:
- Min-Max Scaler
    - Muito sensível a outliers
    - Pode distorcer os dados rescalados por conta do peso dos outliers
- Robust Scaler
    - Considera os quartis individualmente
    - Elimina a sensibilidade a outliers

Portanto vamos aplicar o Robust Scaler para as que tem mais outliers e o Min-Max Scaler para as que tem menos outliers.

##### Robust Scaler

Vamos selecionar as colunas numéricas para aplicar o Robust Scaler:

In [ ]:
#select all numeric columns
# numerical_data = df3.select_dtypes(include=[np.number]).copy()
numerical_data = df3.select_dtypes(include=['int64', 'int32', 'float64', 'float32']).copy()
numerical_data.head()

In [ ]:
numerical_data.shape

In [ ]:
numerical_columns = numerical_data.columns
numerical_columns

Vamos rescalar as colunas numéricas:


Vamos verificar os outliers dessas colunas:

In [ ]:
rs = RobustScaler()
mms = MinMaxScaler()

# rescale robust
target1 = ""
target2 = ""
numerical_data[target1] = rs.fit_transform(numerical_data[[target1]].values)
#pickle.dump(rs, open('..//..//..//Data//Scalers/target1.pkl', 'wb'))

# rescale minmax
numerical_data[target2] = mms.fit_transform(numerical_data[[target2]].values)
#pickle.dump(mms, open('..//..//..//Data//Scalers/target2.pkl', 'wb'))

Neste caso os scalings não vão se apresentar diretamente no dataset, mas vão ser aplicados nele durante o processo de treinamento do modelo. Eles estão sendo salvos como arquivo pickle para serem chamados no próximo passo.

#### Categorical Variable Treatment

##### Encoding

Se tivermos dados categóricos, podemos transformá-los em dados numéricos para o modelo entender e processar a previsão.

Tipos de encoding:
- One-Hot Encoding
    - Transformação de natureza usada com dados de classificação específica, como tipos, tamanhos e cores
- Label Encoding
    - Transformação de natureza usada com dados não cíclicos e específicos, como nomes de estados e cidades
- Ordinal Encoding
    - Transformação de natureza usada com dados sequenciais, como classes, graus de importância, etc.

##### One-Hot Encoding

Vamos selecionar as colunas categóricas para aplicar o One-Hot Encoding:

In [ ]:
categorical_data = df3.select_dtypes(exclude=['int64', 'int32', 'float64', 'float32']).copy()
categorical_data.head()

Sobraram muitas datas como tipo 'object', mas datas não são consideradas categóricas e sim dados cíclicos ou temporais. Portanto vamos dropar as colunas que apresentam datas e ficar apenas com as colunas categóricas.

In [ ]:
remaining_columns = categorical_data.columns
remaining_columns

In [ ]:
remaining_columns = Flexlist(remaining_columns)
type(remaining_columns)

In [ ]:
temporal_columns = remaining_columns[[0, 4, 5, 6]]
temporal_columns

In [ ]:
categorical_columns = remaining_columns[[1,2,3]]
categorical_columns

Vamos ver o resultado ao dropar as colunas temporais:

In [ ]:
categorical_data.drop(temporal_columns, axis=1, inplace=True)
categorical_data.head()

Agora vamos realizar o One-Hot Encoding:

In [ ]:
categorical_data = pd.get_dummies(categorical_data, prefix = ['target_name'], columns = ['target_column_name'])
categorical_data.sample(10)

Esse é um dos tipos de encoding que vemos ser aplicado diretamente no dataset.

##### Label Encoding

In [ ]:
le = LabelEncoder()

categorical_data['store_type'] = le.fit_transform(categorical_data['store_type'])
#pickle.dump(le, open('..//..//..//Data//Encoders/store_type_encoder.pkl', 'wb'))

Neste caso o encoding não vai se apresentar diretamente no dataset, mas vai ser aplicado nele durante o processo de treinamento do modelo. Ele está sendo salvo como arquivo pickle para ser chamado no próximo passo.

##### Ordinal Encoding

In [ ]:
assortment_dict = {'Basic': 1, 'Extra': 2, 'Extended': 3}
categorical_data['assortment'] = categorical_data['assortment'].map(assortment_dict)
categorical_data.sample(10)

In [ ]:
categorical_data.shape

Neste caso o encoding se apresenta diretamente no dataset.

#### Cyclic Variable Treatment

Vamos aplicar o encoding de Seno e Cosseno para as colunas com dados cíclicos ou temporais:

In [ ]:
temporal_data = df3[[*temporal_columns,'day','month','day_of_week','week_of_year']].copy()
temporal_data.head()

In [ ]:
temporal_data.shape

In [ ]:
temporal_data.head()

Neste caso o encoding se apresenta diretamente no dataset, criando novas colunas e alterando os dados a partir das colunas originais.

#### Target Variable Treatment

Vamos aplicar o encoding logarítmico para a variável resposta:

In [ ]:
numerical_data['sales'] = np.log1p(numerical_data['sales'])
numerical_data['sales'].head()

Neste caso o encoding se apresenta diretamente no dataset.

#### Encoded Dataset

Agora vamos fundir os três datasets que criamos durante o rescaling e encoding das variáveis para obter o dataset tratado para o treinamento do modelo:

Vamos conferir os formatos de cada dataset para assegurar que nenhuma **linha** foi adicionada, pois isso tornaria impossível a fusão dos datasets já que perderíamos o índice correto para fundir.

A idéia do Feature Engineering no geral é aumentar o número de **colunas** para encodar as variáveis categóricas e criar rescalers para aplicação nas variáveis numéricas.

In [ ]:
formats = get_feature_engineering_formats(  [numerical_data, categorical_data, temporal_data],
                                            ['numerical_data', 'categorical_data', 'temporal_data'])
formats

##### Dataset Fusion

Com os formatos compatíveis e mesmo número de linhas, vamos fazer a fusão dos datasets:

In [ ]:
encoded_dataset = get_encoded_dataset([numerical_data, categorical_data, temporal_data])
encoded_dataset.shape

In [ ]:
encoded_dataset.head()

In [ ]:
non_eligible_cols = ['week_of_year', 'day', 'month', 'day_of_week', 'promo_since', 'competition_since', 'year_week' ]
encoded_dataset = encoded_dataset.drop(non_eligible_cols, axis=1)

In [ ]:
encoded_dataset.head()

In [ ]:
encoded_dataset.shape

Graças ao encoding e rescaling, ganhamos uma quantidade considerável de colunas no dataset tratado, mas filtramos uma última vez para remover colunas redundantes que foram usadas para criar outras.

##### Encoded Dataset Storage

Vamos salvar o dataset tratado para carregar novamente quando necessário

In [ ]:
save_dataset(encoded_dataset, name = "Dataset_Encoded")
#test = load_dataset('..//..//..//Data//Dataset//Dataset_Encoded.csv')
#test.shape

In [ ]:
#test.head()

### Feature Selection

Agora vamos selecionar as variáveis que vamos utilizar para o treinamento do modelo, o principal objetivo é eliminar variáveis linearmente dependentes de outras, pois quanto mais colunas mais confuso o modelo pode ficar em chegar a uma conclusão, e informações redundantes não ajudam nisso.

Para realizar o Feature Selection existem alguns métodos:
- PCA - Principal Component Analysis
- Lasso - Least Absolute Shrinkage and Selection Operator
- Random Forest - Random Forest Classifier
- Extrative Feature Selection - Extrative Feature Selection
- Feature Importance - Feature Importance


In [ ]:
# training and test dataset for Boruta
#need to pass numpy arrays, not dataframes
#for this we use the .values attribute and the ravel method
#X_train_n = X_train.values
#y_train_n = y_train.values.ravel() #ravel method returns a flattened array

# define RandomForestRegressor
#rf = RandomForestRegressor(n_jobs=-1)

# Train Boruta
#boruta = BorutaPy(rf, n_estimators='auto', verbose=2, random_state=42).fit(X_train_n, y_train_n)
#feature_selection_cols = boruta.support_.tolist() #returns a dataframe with the selected columns

#### Best Features

Com as colunas mais importantes separadas, vamos verificar se alguma coluna não ficou de fora quando comparado com nossas conclusões na fase da análise exploratória, e também ver o quanto as colunas escolhidas pelo algoritmo se aproximam das conclusões da análise.

#### Final Dataset Storage

Vamos salvar o dataset final para carregar novamente quando necessário.

In [ ]:
encoded_dataset[feature_selection].head()

In [ ]:
model_dataset = encoded_dataset[feature_selection]

In [ ]:
model_dataset.shape

In [ ]:
save_dataset(model_dataset, name = "Dataset_Model")

In [ ]:
import gc
gc.collect()

Com isso temos nosso dataset final salvo em um arquivo CSV.

## Model Building

In [ ]:
Image(filename='..\\..\\Assets\\Images\\Model Building.png', width=200, height=100)

A parte de Model Building é composta principalmente por três partes:
- Train-Test Splitting
   - Vamos separar o dataset em treino e teste, descartando as variáveis resposta
- Model Selection
    - Baseline Model
        - Vamos criar um modelo base para comparar com o modelo que vamos construir
    - Model Type Selection
        - Vamos escolher o melhor tipo de modelo para o problema
        - Escolhemos alguns modelos dessa classe para comparar com o modelo base
- Model Training
    - Baseline Training
        - Vamos treinar os modelos escolhidos normalmente e verificar os resultados
    - Cross Validation Training
        - Vamos treinar os modelos escolhidos cada um com o método de Cross Validation para ver seu desempenho geral no dataset
        - A divisão em treino e teste feita por nós é apenas um dos jeitos de se fazer a divisão do dataset, o modelo pode performar muito bem ou muito mal na divisão que fizemos
        - Por isso que executamos o Cross Validation, para avaliar o desempenho do modelo no dataset geral, com várias possibilidades de divisões em treino e teste sendo testadas e calculando a média do desempenho em cada uma

### Model Building Checkpoint

Fazendo um checkpoint, carregamos o dataframe para uma nova variável, isolando os resultados obtidos nesta seção dentro dela e evitando propagações de erros que requeiram a reexecução do notebook inteiro.

In [ ]:
model_dataset = load_dataset('Dataset_Model')
model_dataset.shape

In [ ]:
model_dataset.dtypes

In [ ]:
model_dataset["date"] = pd.to_datetime(model_dataset["date"])
model_dataset.dtypes

### Train-Test Splitting

Vamos separar o dataset final em treino e teste, descartando as variáveis resposta, e também em treino de Cross Validation, neste caso preservando as variáveis resposta.

Vamos especificar qual é a variável resposta quando o Cross Validation for usado.

In [ ]:
# training dataset
X_train = model_dataset[model_dataset['date'] < '2015-06-05']
y_train = X_train['sales']
# test dataset
X_test = model_dataset[model_dataset['date'] >= '2015-06-05']
y_test = X_test['sales']
# Cross validation training/testing dataset, uses all the features, including the target variable
# We'll specify the target variable in the cross val function
X_Cross = model_dataset[cross_val_cols]
# Minimun and maximum date for the Train and Test dataset
Train_min_date, Train_max_date = X_train['date'].min(), X_train['date'].max()
Test_min_date, Test_max_date = X_test['date'].min(), X_test['date'].max()
# Drop the target variable 'sales' and future variable 'date' from the XTrain dataframe
# In order to do not train with it
# Also drop every other column that is not needed for the model, using the filter 'feature_selection_cols'
X_train = X_train[feature_selection_cols]
X_test = X_test[feature_selection_cols]

In [ ]:
print(f'Training Min Date: {Train_min_date}')
print(f'Training Max Date: {Train_max_date}')

print(f'\nTest Min Date: {Test_min_date}')
print(f'Test Max Date: {Test_max_date}')

Com isso temos 3 datasets:
- Treino
    - X_train: Variáveis de treino
    - y_train: Variável única de resposta do treino
    - Estes datasets passam pelo treinamento do modelo normal
- Teste
    - X_test: Variáveis de teste
    - y_test: Variável única de resposta do teste
    - Estes datasets não passam pelo treinamento do modelo, ficando intocados até o final
- Cross Validation
    - X_Cross: Variáveis de treino para o Cross Validation, incluindo a variável resposta e a variável de data
    - A variável de data tem de estar presente pois temos uma série temporal como problema
    - Este dataset passa pelo treinamento do modelo em Cross Validation

### Model Selection

Vamos criar o modelo de baseline e escolher os modelos com os quais vamos trabalhar no dataset, dentro desta fase teremos dois tipos de treinamento:
- Baseline Training
    - Vamos treinar os modelos de modo normal, com os datasets de treino e teste
- Cross Validation Training
    - Vamos treinar os modelos no modo de Cross Validation, com o dataset de crossval, e verificar o desempenho real no dataset

#### Baseline Training

##### Baseline Model

Nosso modelo de baseline será o Average Model, que retorna a média de vendas que uma loja teve em todo o período registrado.

In [ ]:
baseline_X = X_test.copy()
baseline_X['sales'] = y_test.copy()

# prediction
baseline_Y = baseline_X[['store', 'sales']].groupby('store').mean().reset_index().rename(columns={'sales': 'predictions'})
baseline_Y.head()

Com a predição de média feita, vamos juntar os datasets de treino e teste, e verificar o desempenho do modelo.

In [ ]:
baseline_model = pd.merge(baseline_X, baseline_Y, how='left', on='store')
y_pred_baseline = baseline_model['predictions']

# performance
baseline_result = model_metrics('Average Model', y_test, y_pred_baseline, 'Exponential')
baseline_result

##### Linear Regression Model

In [ ]:
# model
#lnr_model = LinearRegression().fit(X_train, y_train)

#prediction
#ypred_lnr = lnr_model.predict(X_test)

#performance
#lnr_result = model_metrics('Linear Regression', y_test, ypred_lnr, 'Exponential')
#save_model(lnr_model, 'Linear Regression')
#lnr_result

##### Lasso Regression Model

In [ ]:
# model
#lsr_model = Lasso(alpha=0.01).fit(X_train, y_train)

# prediction
#ypred_lsr = lsr_model.predict(X_test)

# performance
#lsr_result = model_metrics('Lasso Regression', y_test, ypred_lsr, 'Exponential')
#save_model(lsr_model, 'Lasso Regression')
#lsr_result

##### Random Forest Regressor Model

In [ ]:
# model
#rfr_model = RandomForestRegressor(n_estimators=100, n_jobs=-1, random_state=42).fit(X_train, y_train)

# prediction
#ypred_rfr = rfr_model.predict(X_test)

# performance
#rfr_result = model_metrics('Random Forest Regressor', y_test, ypred_rfr, 'Exponential')
#save_model(rfr_model, 'Random Forest Regressor')
#rfr_result

##### XGBoost Regressor Model

In [ ]:
# model
xgbr_model = XGBRegressor(objective='reg:squarederror', n_estimators=100, eta=0.01, max_depth=10,
                            subsample=0.7, colsample_bytee=0.9).fit(X_train, y_train)

# prediction
ypred_xgbr = xgbr_model.predict(X_test)

# performance
xgbr_result = model_metrics('XGBoost Regressor', y_test, ypred_xgbr, 'Exponential')
#save_model(xgbr_model, 'XGBoost Regressor')
xgbr_result

In [ ]:
#import gc
#gc.collect()

#### Cross Validation Training

In [ ]:
Image(filename='..\\..\\Assets\\Images\\Time_series_training.png', width=1000, height=500)

##### Linear Regression Model

In [ ]:
#lnr_result_cv = crossval_time_series(X_Cross, 5, 8, 'Linear Regression', lnr_model, (['date', 'sales'],['sales']), verbose = True)

#lnr_result_cv

##### Lasso Regression Model

In [ ]:
#lsr_result_cv = crossval_time_series(X_Cross, 5, 8, 'Lasso Regression', lsr_model, (['date', 'sales'],['sales']), verbose = True)

#lsr_result_cv

##### Random Forest Regressor Model

In [ ]:
#rfr_result_cv = crossval_time_series(X_Cross, 5, 8, 'Random Forest Regressor', rfr_model, (['date', 'sales'],['sales']), verbose = True)

#rfr_result_cv

##### XGBoost Regressor Model

In [ ]:
#xgbr_result_cv = crossval_time_series(X_Cross, 5, 8, 'XGBoost Regressor', xgbr_model, (['date', 'sales'],['sales']), verbose = True)

#xgbr_result_cv

In [ ]:
#import gc
#gc.collect()

#### Models Performance

##### Baseline Training Comparison

In [ ]:
#modelling_result = pd.concat([baseline_result, lnr_result, lsr_result, rfr_result, xgbr_result])
#modelling_result

In [ ]:
Image(filename='..\\..\\Assets\\Images\\Baseline_Training_Comparison.png', width=700, height=180)

##### Cross Validation Training Comparison

In [ ]:
#modelling_result_cv = pd.concat([lnr_result_cv, lsr_result_cv, rfr_result_cv, xgbr_result_cv])
#modelling_result_cv

In [ ]:
Image(filename='..\\..\\Assets\\Images\\Cross_Validation_Training_Comparison.png', width=900, height=170)

## Model Evaluation

In [ ]:
Image(filename='..\\..\\Assets\\Images\\Model Evaluation.png', width=200, height=100)

Nesta fase avaliamos o desempenho dos modelos escolhidos e alteramos seus parâmetros para maximizar o desempenho. Também escolhemos o melhor modelo para usar como modelo final desta análise.

A Model Evaluation é composta de três partes:
- Model Comparison
    - Vamos comparar os modelos escolhidos com o modelo base e verificar qual tem o melhor desempenho geral no dataset, resultado obtido no desempenho do cross validation
- Model Hyperparameter Fine Tuning
    - Vamos alterar os parâmetros do modelo escolhido para melhorar o desempenho de acordo com algum algoritmo de otimização de parâmetros
- Final Model Selection
    - Vamos treinar o melhor modelo com os parâmetros finais escolhidos
    - Este modelo será usado para prever os dados de teste
    - Este modelo será salvo em um arquivo binário .pkl para uso posterior

### Model Evaluation Checkpoint

Fazendo um checkpoint, copiamos o dataframe para uma nova variável, isolando os resultados obtidos nesta seção dentro dela e evitando propagações de erros que requeiram a reexecução do notebook inteiro.

In [ ]:
# df5 = df4.copy()
#print("Checkpoint successful")

In [ ]:
# df5.head()

### Model Hyperparameter Fine Tuning

Nesta fase buscamos encontrar a melhor combinação de parâmetros possíveis para maximizar o aprendizado e o desempennho do modelo.

A fase de hyperparameter tuning pode ser feita de três formas:
- Random Search
    - Define valores aleatórios para os parâmetros do modelo
    - Veloz, porém não testa todas as combinações de parâmetros possíveis
- Grid Search
    - Define todas as combinações possíveis de parâmetros para o modelo
    - Extremamente lento, mas testa todas as combinações de parâmetros possíveis
    - Sempre chega ao melhor resultado possível
- Bayesian Search
    - Intermediário entre os dois, é mais lento que o Random Search, mas mais rápido que o Grid Search
        - Define os valores de parâmetros do modelo com base na teoria de Bayes
    - Busca os melhores parâmetos futuros com base nos resultados de cada treinamento anterior

#### Random Search

Vamos utilizar o Random Search para encontrar a melhor combinação de parâmetros para o modelo. 
- Um bom número de ciclos de busca é 10
- Quanto maior o número de estimadores, mais demorado o processo de busca

In [ ]:
param = {
    'n_estimators': [150, 170, 250, 300, 350],#[1500, 1700, 2500, 3000, 3500]
    'eta': [0.01, 0.03],
    'max_depth': [3, 5, 9],
    'subsample': [0.1, 0.5, 0.7],
    'colsample_bytree': [0.3, 0.7, 0.9],
    'min_child_weight': [3, 8, 15]
        }

cycles = 3 #3

In [ ]:
random_search_tms(cycles, X_Cross, 5, 8, 'XGBoost Regressor tuned', (['date', 'sales'],['sales']), verbose = True)

### Hyperparameter Evaluation

A melhor métrica para definir o modelo que mais se adequa ao dataset de modo genérico (com cross validation) é o RMSE, ou seja, o erro quadrático médio (root mean squared error).
- No caso desta análise, com 3 ciclos de busca vimos que no primeiro ciclo o modelo já obteve o melhor resultado, com o menor RMSE entre todos os modelos testados
- Com RMSE de **2588.24 +/- 96.08**, o menor entre todos (4678.03 e 3941.8)
    - E com os parâmetros:
    - **n_estimators:** 350
    - **eta:** 0.03
    - **max_depth:**	3
    - **subsample:** 0.5
    - **colsample_bytree:** 0.9
    - **min_child_weight:** 15

In [ ]:
Image(filename='..\\..\\Assets\\Images\\Random_search_3_cycles_training.png', width=1500, height=160)

### Final Model

O modelo final só surge após o treinamento do modelo com todos os parâmetros finais otimizados.
- Os testes de desempenho já foram feitos na fase de Cross Validation 
    - O modelo X foi o melhor por ter o menor RMSE entre todos
- Os parâmetros finais foram descobertos na fase de Random Search
- Ele será treinado e refinado com esses parâmetros escolhidos
- Vamos fazer uma predição para o dataset de teste (y_test) para testar seu desempenho final de negócio
- Então vamos salvar o modelo em um arquivo binário .pkl para uso posterior no Deployment

In [ ]:
param_tuned =   {
                'objective':'reg:squarederror',
                'n_estimators': 3000,
                'eta': 0.03,
                'max_depth': 5,
                'subsample': 0.7,
                'colsample_bytree': 0.7,
                'min_child_weight': 3 
                }

In [ ]:
# model
model_xgbr_tuned = XGBRegressor(**param_tuned).fit(X_train,y_train)

In [ ]:
# saving tuned model
save_model(model_xgbr_tuned, 'XGBoost_Regressor_Tuned')

In [ ]:
import gc
gc.collect()

### Metrics Interpretation

Nesta fase vamos entender quais são as métricas de erro usadas em Data Science e quais os seus usos principais.

Temos 5 principais métricas de erro para avaliar o desempeho de um modelo em Data Science, e 3 curvas de desempenho:

#### Business Metrics

- MAE - Mean Absolute Error
    - Atribui peso igual a todos os erros
    - Robusto na presença de outliers, não afetam tanto o resultado
    - Fácil entendimento pelo time de negócios
    - Toda vez que fizermos uma predição, estamos errando na média em 500 dólares pra cima ou pra baixo
- MAPE - Mean Absolute Percentage Error
    - Erro percentual da predição, toda vez que fizermos uma predição, estamos errando em pelo menos 10% do valor predito
    - Mostra o quão longe a predição está do valor real, na média, em porcentagem
- Não podem ser usados quando a variável resposta contém zeros, precisando ser filtrados antes do cálculo

#### Model Metrics

- Sensíveis a outliers, que afetam bastante o resultado
- RMSE - Root Mean Squared Error
    - Atribui maior peso aos erros maiores
    - Por ser sensível a toda a gama de dados disponíveis, é considerado uma das melhores métricas de desempenho
    - Quanto menor, melhor
    - Ideal para medir a performance dos modelos de machine learning
- RMSPE - Root Mean Squared Percentage Error
    - Erro RMSE em porcentagem
- MPE - Mean Percentage Error
    - É uma métrica útil para avaliar pra qual lado mais pende a imprecisão do modelo, para cima ou para baixo nos resultados de negócio, ou seja, se falha mais prevendo o pior ou o melhor caso.
    - Diz apenas se o modelo está mais pendente para subestimar ou superestimar o valor real
    - Se for positivo, o modelo está superestimando, se for negativo, está subestimando o valor real
    - Não pode ser usado quando a variável resposta contém zeros, precisando ser filtrados antes do cálculo
- RMSEE - Root Mean Squared Error of Estimation
    - Erro quadrático médio da predição, ou seja, essa métrica nos dá o erro da diferença de valores entre a predição e o valor real
    - Usado para refinar a precisão das predições
- Curvas de Desempenho
    - ROC - ROC Curve
    - LIFT - Lift Curve
    - AUC - Area Under the ROC Curve

In [ ]:
model_dataset = load_dataset('Dataset_Model') 

In [ ]:
model_dataset.head()

In [ ]:
model_dataset[['sales']].head()

In [ ]:
# training dataset
X_train = model_dataset[model_dataset['date'] < '2015-06-05']
y_train = X_train['sales']
# test dataset
X_test = model_dataset[model_dataset['date'] >= '2015-06-05']
y_test = X_test['sales']

X_train = X_train[feature_selection_cols]
X_test = X_test[feature_selection_cols]

In [ ]:
final_model = load_model('XGBoost_Regressor_Tuned')

In [ ]:
# prediction
y_pred = final_model.predict(X_test)

# performance
final_model_result = model_metrics('XGBoost Regressor Tuned', y_test, y_pred, 'Exponential')

#final_result = pd.DataFrame()

In [ ]:
#concatenate the dataframes horizontally
#final_result = pd.concat([final_result, modelling_result_cv], axis=1)
#final_result = pd.concat([final_result, final_model_result])
#final_result.set_index('Model Name', inplace=True)

In [ ]:
#final_result
final_model_result

#### MAE e MAPE

Primeiro vamos ver qual o máximo e mínimo da nossa variável resposta para descobrir qual sua faixa(range) de valores.

In [ ]:
print(np.expm1(y_test).min(), np.expm1(y_test).max())

Range da variável resposta:

In [ ]:
np.expm1(y_test).max()-np.expm1(y_test).min()

In [ ]:
#final_result
final_model_result

Se temos um MAE de 779, quanto % isso representa do nosso range da variável resposta?

In [ ]:
print(str(round(779/41000 * 100, 2)) + '%')

Se temos um MAE de 665, quanto % isso representa do nosso valor médio variável resposta?

In [ ]:
np.expm1(y_test).mean()

In [ ]:
print(str(round(779/np.expm1(y_test).mean() * 100, 2)) + '%')

Em média fazemos vendas de 7016 dólares, e pra cada predição que fizermos, temos um erro de 665 dólares, que é cerca de 11% deste valor.

O MAPE é a representação em porcentagem do MAE.

#### RMSE e MPE

Métricas mais usadas para avaliar o desempenho do modelo

In [ ]:
#final_result
final_model_result

In [ ]:
print(str(round(1096/np.expm1(y_test).mean() * 100, 2)) + '%')

Isso signifca que, considerando todos os valores do dataset, inclusive os outliers, o modelo está errando em 15.62% dos casos na sua previsão do valor real.

O que significa que está acertando em 84.38% dos casos, considerando apenas esta métrica.

O MPE é negativo, o que indica que o valor predito tende a ser maior do que o valor real, ou seja, o ponto fraco do modelo é no melhor caso.

# Deployment Phase

In [ ]:
Image(filename='..\\..\\Assets\\Images\\Deployment Phase.png', width=250, height=100)

## Visualization and Dashboard

In [ ]:
Image(filename='..\\..\\Assets\\Images\\Visualization and Dashboard.png', width=200, height=100)

### Visualization and Dashboard Checkpoint

Fazendo um checkpoint, copiamos o dataframe para uma nova variável, isolando os resultados obtidos nesta seção dentro dela e evitando propagações de erros que requeiram a reexecução do notebook inteiro.

In [ ]:
# df7 = df6.copy()
#print("Checkpoint successful")

In [ ]:
# df7.head()

### Performance assessment

Como avaliamos a performance do modelo contra a performance do negócio? Isto é, conseguimos melhorias de desempenho com o modelo? Como aferir isso? É nesta fase que verificamos se todo o treinamento e refinamento do modelo valeram a pena em relação ao que já tinhamos previamente do negócio.

#### Performance Features

Para avaliar a performance do modelo vamos criar algumas colunas extras no dataset final

In [ ]:
model_dataset.shape, y_pred.shape

Como podemos ver, o dataset final tem muito mais linhas do que o de predição, isso porque a predição só trata das últimas 8 semanas, na coluna Data do dataset final.
para podermos fundir a coluna de predição com o dataset final e formar o dataset de visualização vamos ter que descartar as linhas as quais não temos dados de predição, ou seja, as linhas que foram usadas para treino.

Isso equivale a chamar nosso novo dataset de visualização como o dataset que fora usado para treino nos passos anteriores.

Vamos checar a quantidade de linha dos dois para confirmar.

In [ ]:
model_dataset.shape, y_pred.shape, X_test.shape

Como suspeitamos, as linhas são idênticas na coluna de predição e de teste. Então nosso dataset de visualização precisa ser exatamente igual a coluna que foi de teste dos modelos. 
É o que faremos agora.
Este foi o filtro que usamos para gerar a coluna de teste:

In [ ]:
print("X_test = model_dataset[model_dataset['date'] >= '2015-06-05']")


E é o que vamos usar agora no model_dataset para criar o de visualização.

In [ ]:
# viz_dataset
viz_dataset = model_dataset[model_dataset['date'] >= '2015-06-05'] #previously X_test

In [ ]:
viz_dataset[['sales']].head()

In [ ]:
y_pred.shape, viz_dataset.shape

Vamos regredir o scaling da variável resposta para o valor real, pois assim a visualização dos gráficos não ficará distorcida e com valores baixos demais.

Nós rescalamos quando vamos treinar e predizer, mas retornamos ao valor real da variável resposta quando vamos avaliar métricas do modelo e visualizar o desempenho final.

In [ ]:
# rescale target variable
viz_dataset['sales'] = np.expm1(viz_dataset['sales'])
viz_dataset['predictions'] = np.expm1(y_pred)
viz_dataset['error_pred'] = viz_dataset['sales'] - viz_dataset['predictions']
viz_dataset['error_rate'] = viz_dataset['predictions']/viz_dataset['sales']

In [ ]:
# get predictions
viz_dataset[['sales','predictions','error_pred','error_rate']].sample(10)

In [ ]:
viz_cols = Flexlist(viz_dataset.columns.tolist())
print(viz_cols, type(viz_cols) )

In [ ]:
list_indexer(viz_cols)

In [ ]:
ordered_vis_cols = viz_cols[[21,0,1,2,3,4,22,23,24,25]]
print(ordered_vis_cols)

#### Model Performance

Para isto, o poblema de negócio já deve ter algum tipo de métrica de previsão anterior, por ela será possível avaliar o desempenho do modelo. Como neste caso não temos algo vindo diretamente do negócio, vamos usar o modelo de baseline assumindo que ele já veio pronto com o problema e também as métricas do modelo para prever incrementos de receita junto com melhor e pior caso.

##### Baseline vs Model Performance

In [ ]:
# dataframe with baseline_results and tuned_results

model_performance = pd.concat([baseline_result, final_model_result], axis=0)
model_performance.sort_values(by='RMSE', ascending=False, inplace=True)
model_performance.set_index('Model Name', inplace=True)
model_performance

In [ ]:
plt.suptitle('Model Metrics')
plt.subplot(2, 2, 1)
plt.title('RMSE Comparison')
sns.barplot(x = model_performance.index, y = 'RMSE', data = model_performance)
plt.subplot(2, 2, 2)
plt.title('MAE Comparison')
sns.barplot(x = model_performance.index, y = 'MAE', data = model_performance)
plt.subplot(2, 2, 3)
plt.title('MAPE Comparison')
sns.barplot(x = model_performance.index, y = 'MAPE', data = model_performance)
plt.subplot(2, 2, 4)
plt.title('MPE Comparison')
sns.barplot(x = model_performance.index, y = 'MPE', data = model_performance)
plt.show()

Vamos ver a distribuição do erro por predição:

In [ ]:
plt.figure(figsize = (20,10))
plt.suptitle('Error by prediction distribution')
sns.distplot(viz_dataset['error_pred'])
plt.show()

#### Model Performance in Business

#### Business Performance Gain

Aqui vamos ver o que o modelo fez para melhorar o desempenho do negócio, fazendo uma previsão geral de receita para toda a rede de lojas.

Temos o melhor e pior cenário de vendas de toda a rede de lojas, somando todas as lojas, nas próximas 8 semanas.

Como os dados nos mostram, o modelo final é consideravelmente melhor que o modelo de baseline, chegando a cortar em quase metade o RMSE quando comparado com o mesmo, e o superando também em todas as outras métricas.

Com isto concluimos que após a análise feita, as features selecionadas, o modelo treinado e seus parâmetros refinados, que o modelo final tem maior taxa de sucesso que o método usado anteriormente na predição de vendas.

## API development

In [ ]:
Image(filename='..\\..\\Assets\\Images\\Webapp Deployment.png', width=200, height=100)

### API development Checkpoint

Fazendo um checkpoint, copiamos o dataframe para uma nova variável, isolando os resultados obtidos nesta seção dentro dela e evitando propagações de erros que requeiram a reexecução do notebook inteiro.

In [ ]:
# df6 = df5.copy()
#print("Checkpoint successful")

In [ ]:
# df6.head()

#### Prediction Class

Na classe em produção vão ser incluídas as fases:
- Data Collection
- Data Cleaning
    - Todas as fases, pois modificam o dataset diretamente
- Feature Engineering
    - Data Preparation
        - Numerical Variable Treatment
        - Categorical Variable Treatment
        - Cyclic Variable Treatment
        - Target Variable Treatment
        - Encoded dataset
    - Feature Selection
        - Best Features
        - Final Dataset Storage
- Model Building
    - Train-Test Splitting
    - Model Selection
- Model Evaluation
    - Model Hyperparameter Fine Tuning
    - Final Model
- API
    - FastAPI
    - Routes
    - Predict route
    - Predict function
    - Model import with pickle
    - jsonify function
    - pipeline function

#### API Handler

Nesta seção vamos criar o handler que vai ser responsável por fazer a chamada da API.
Nossa API terá dois endpoints:
- /predict
    - Recebe um JSON o número direto da loja a se prever as vendas
- /telegram
    - Recebe um JSON com os dados da requisição ou o número direto da loja a se prever as vendas

In [ ]:
import os
import pickle
import pandas as pd
import requests
import json
from flask import Flask, request, Response, render_template, abort, redirect
#from folder.file import class
from Store_Sales_Analysis import Store_Sales_Analysis
# get token from .env file
token = os.environ.get('Token')

def set_webhook_telegram(url = None, token = None):
    url = url + token
    url = url + '/setWebhook?url=' + url
    api_call = requests.post(url)
    print(f'Status Code {api_call.status_code}')
    return None

def send_message(chat_id = None, text = None, token = None):
    url = f'https://api.telegram.org/bot{token}/'
    url = url + f'sendMessage?chat_id={chat_id}'
    
    api_call = requests.post(url, json = {'text': text })
    print(f'Status Code {api_call.status_code}')
    
    return None

#middleware
def get_prediction(data):
    # API Call
    # makes an API call to /predict endpoint
    url_prod = 'https://andrew-store-sales-analysis.herokuapp.com/predict'
    dev_url = 'http://127.0.0.1:8000/predict'
    #port = os.environ.get('PORT', 8000)
    #'http://localhost:5000/register'
    url = url_prod
    header = {'Content-type': 'application/json'}
    #data = data
    
    api_call = requests.post(url, data = data, headers = header)
    
    print(f'Status Code {api_call.status_code}')
    
    #return str(str(api_call.status_code) + ' ' + str(api_call.text) + 'HERE')
    prediction = pd.DataFrame(api_call.json(), columns = api_call.json()[0].keys())
    #return api_call.json()
    return prediction

def load_model(model_name):
    # loading model in readbytes mode
    current_path = os.path.dirname(os.path.abspath(__file__))
    model = pickle.load(open(current_path + model_name + '.pkl', 'rb'))
    
    return model

def load_dataset(store_id):
    # loading test dataset
    try:
        df_test_raw = pd.read_csv('test.csv')
        df_store_raw = pd.read_csv('store.csv')
        
    except Exception as e:
        print('error loading datasets')
        print(e)
    # merge test dataset + store
    df_test = pd.merge(df_test_raw, df_store_raw, how = 'left', on = 'Store')
    
    # choose store for prediction
    df_test = df_test[df_test['Store'] == store_id]
    
    if not df_test.empty:
        # remove closed days
        df_test = df_test[df_test['Open'] != 0]
        df_test = df_test[~df_test['Open'].isnull()]
        df_test = df_test.drop('Id', axis = 1)
        # convert Dataframe to json
        data = json.dumps(df_test.to_dict(orient = 'records'))
        
    else:
        data = 'error'
        
    return data

def parse_message(message = None):
    
    chat_id = message['message']['chat']['id']
    store_id = message['message']['text']
    store_id = store_id.replace('/', '')
    # if store_id is /start send store_id = '/start' to treat that later
    try:
        store_id = int(store_id)
        
    except ValueError:
        print('Store id needs to be a number')
        store_id = 'error'
        
    return chat_id, store_id

def get_response(response, error = None, endpoint = None,  message_chat_id = None, message_text = None):
    
    #implement with match case with python 3.10
    if response == 0: #debug one
        message =  {'Keys': 'are ok',
                    'message_chat_id': message_chat_id,
                    'message_text': message_text,
                    'endpoint': endpoint}
    elif response == 1:
        message =  {"hello": r"Greetings, I am a telegram bot",
                    "error": r"There are missing keys in the request",
                    "instruction": r"Please send a json object with the following keys:",
                    "message": r"{ chat: {id:chat_id, type:chat_type}, text:/some_text}",
                    "goodbye": f"This proofs you could access the endpoint {endpoint}"}
    elif response == 2:
        message = {"error": "No data received, json empty, please send a valid json object"}
    elif response == 3:
        message = {"error": "No json received, data header doesn't indicate a json object, please send a json object"}
    elif response == 4:
        message = {"error": "Method not allowed"}
    elif response == 5:
        message = {"success": "Message sent to telegram chat successfully with prediction"}
    elif response == 6:
        message =  {"success": "Message sent to telegram chat successfully but no prediction was made",
                    "error": error}
    elif response == 7:
        # greetings message when /start is called
        message = '''Hello! I am a telegram bot.
                    I can predict the sales of a store.
                    Please send me a store number or a table with values to predict the sales. 
                    '''
        return message
    
    message = json.dumps(message)
    
    return message

def check_json(data = None, endpoint = None, method = None):
    
    # check if request.content_type is json or if it is empty
    if (data.content_type != '' and data.is_json): #check if data header indicates json and is not empty
        # remove empty check in case telegram doesn't send correct headers with json type
        # perhaps remove it entirely if telegram doesn't send headers at all
        received_json = data.get_json()
        # log json received
        if any(received_json):
            # check if required keys are in the json
            # can add more later if needed
            try:
                message_chat_id = received_json['message']['chat']['id']
                message_text = received_json['message']['text']
                # log json not empty, keys are in the json
                
            except Exception as e:
                # only error that can occur here is if some key doesn't exist
                # log json not empty, error key not found
                # return instructions
                abort(Response(get_response(1, None, endpoint), status = 400, mimetype = 'application/json'))
                
            finally:
                
                if method == 'GET':
                    # return instructions
                    abort(Response(get_response(1, None, endpoint), status = 400, mimetype = 'application/json'))
                    
                else:
                    
                    # return json
                    return received_json
                # comment when deploying
                # return abort(Response(get_response(0, None, endpoint, message_chat_id, message_text), status = 200, mimetype = 'application/json'))
        else:
            # return error, json is empty
            abort(Response(get_response(2), status = 400, mimetype = 'application/json'))
    else:
        # return error, data header doesn't indicate a json object
        abort(Response(get_response(3), status = 400, mimetype = 'application/json'))

def check_entrypoint(call = None, endpoint = None):
    
    if endpoint in ['/telegram', '/webapp']:
        # external endpoint, need to check if method is GET, POST or other
        if call.method == 'POST':
            #log post method requested
            call_json = check_json(request, endpoint, 'POST')
            
            return call_json
            
        elif call.method == 'GET':
            #log get method requested
            check_json(request, endpoint, 'GET')
            
        else:
            # log other method requested
            # return method not allowed
            abort(Response(get_response(4), status = 400, mimetype = 'application/json'))
            
    elif endpoint == '/predict':
        
        if call.method == 'GET':
            # return instructions
            abort(Response(get_response(1, None, endpoint), status = 400, mimetype = 'application/json'))
        # internal endpoint, no need to check method
        call_json = call.get_json()
        
        return call_json
    
# API initialization
app = Flask(__name__, static_folder='static')

@app.route('/')
@app.route('/home')
def home():
    
    return render_template('index.html')

@app.route('/webapp')
def webapp():
    
    return '<p> Welcome to the webapp </p>'
    # return render_template('webapp.html')

@app.route('/telegram', methods = ['GET', 'POST'])
def telegram_bot():
    
    received_json = check_entrypoint(request, endpoint = '/telegram')
    
    chat_id, store_id = parse_message(received_json)
    #third: command
    if store_id != 'error':
        # loading data
        data = load_dataset(store_id)
        
        # if not returned an empty dataset (string 'error') because store_id doesn't exist
        if data != 'error':
            # prediction call
            prediction_df = get_prediction(data)
            # calculation
            pred_group_df = prediction_df[['store', 'prediction']].groupby('store').sum().reset_index()
            # get store number
            store = pred_group_df['store'].values[0]
            # get prediction value
            prediction = pred_group_df['prediction'].values[0]
            # convert to number
            prediction = float(prediction)
            # send message with store number and prediction in text format to telegram bot chat
            msg = f'Store Number {store} will sell R${prediction:,.3f} in the next 8 weeks'
            
            #print(msg)
            send_message(chat_id, msg, token)
            # return message sent to telegram chat, in this case, success
            return Response(get_response(5), status =200)
        
        else: #command
            error = 'Store Not Available'
            send_message(chat_id, error, token)
            # return message sent to telegram chat, in this case, error
            return Response(get_response(6, error = error), status =200)
        
    else:
        error = 'Store ID must be a number'
        send_message(chat_id, error, token)
        # return message sent to telegram chat, in this case, error
        return Response(get_response(6, error = error), status =200)

@app.route('/predict', methods = ['GET', 'POST'])
def model_predict():
    
    received_json = check_entrypoint(request, endpoint = '/predict')
    
    model = load_model('/static/models/XGBoost_Regressor_Tuned')
    
    # redo this later, strange check (?). Must be a better way to do this
    if isinstance(received_json, dict): # unique example
        
        X_test = pd.DataFrame(received_json, index = [0])
        
    else: # multiple example
        X_test = pd.DataFrame(received_json, columns = received_json[0].keys())
    
    pipeline = Store_Sales_Analysis()
    
    data_cleaned = pipeline.data_cleaning(X_test)
    
    feature_extracted = pipeline.feature_extraction(data_cleaned)
    
    data_prepared = pipeline.data_preparation(feature_extracted)
    
    prediction = pipeline.get_prediction(model, X_test, data_prepared)
    
    #return prediction
    return Response(prediction, status = 200, mimetype = 'application/json')

if __name__ == '__main__':
    port = os.environ.get('PORT', 8000)
    host_prod = os.environ.get('HOST', '0.0.0.0')
    app.run(host = host_prod, port = port)

#### API Tester

Vamos simular uma chamada de API do jeito que ela funcionaria no app final aqui no Notebook de Análise

- Primeiro, carregamos os dataframes com os dados de cada loja.
- Depois, carregamos o dataframe de teste com dados ainda não vistos pelo modelo.

In [ ]:
# loading test dataset
df_test = pd.read_csv("A:\Andrew\Desenvolvimento\Portfolio\Backup\Store Sales Analysis\Development\Data\Dataset\Raw\test.csv")
# merge test dataset + store
df_request = pd.merge(df_test, df_store_raw, how = 'left', on = 'Store')

# choose store for prediction
df_request = df_request[df_request['Store'].isin([40, 22, 41])]

# remove closed days
df_request = df_request[df_request['Open'] != 0]
df_request = df_request[~df_request['Open'].isnull()]
df_request = df_request.drop('Id', axis = 1)
# convert Dataframe to json
data = json.dumps(df_request.to_dict(orient = 'records'))

Agora fazemos a chamada da API, passando o número da loja ou o JSON com os dados da requisição.
- A chamada deve funcionar tanto localmente quanto no Heroku.

In [ ]:
# API Call
url_local = 'http://0.0.0.0:5000/predict'
url_api = 'https://andrew-store-sales-analysis.herokuapp.com/predict'
header = {'Content-type': 'application/json' } 
data = data

response = requests.post(url_api, data = data, headers = header)
print('Status Code {}'.format(r.status_code))
df_response = pd.DataFrame(response.json(), columns = response.json()[0].keys())

In [ ]:
d2 = df_response[['store', 'prediction']].groupby('store').sum().reset_index()

for i in range(len(d2)):
    print('Store Number {} will sell R${:,.2f} in the next 6 weeks'.format(
            d2.loc[i, 'store'], 
            d2.loc[i, 'prediction']))

## Web App

### Web App Checkpoint

### Frontend

#### Timer

In [ ]:
print("Notebook Start:", str(Start).split(' ')[1].split('.')[0])
      #log.info("Notebook Start =", now)
End = datetime.datetime.now() # time object
print("Notebook End:", str(End).split(' ')[1].split('.')[0])
Execution = End - Start
print("Execution Time:", str(Execution).split('.')[0]) 